# Active Scans Report (BMC Discovery)

This notebook fetches discovery run activity from a BMC Discovery appliance using the REST API and loads the results into pandas for inspection and export.

## Requirements

We use `requests` for HTTP, `pandas` for tabular data, and `PyYAML` to read configuration. If they are not installed, uncomment the install cell below.

In [1]:
# %pip install -q tideway pandas pyyaml

import pandas as pd
import yaml
from pathlib import Path
from typing import Any, Dict, Optional
import xml.etree.ElementTree as ET
import tideway


In [2]:
APPLIANCE_NAME: Optional[str] = None
APPLIANCE_INDEX: int = 0


## Select Appliance (optional)

If your `config.yaml` defines multiple appliances under the `appliances:` list, set `APPLIANCE_NAME` to one of their names (recommended) or set `APPLIANCE_INDEX` to pick by position. Leave both as-is to default to the first appliance.

In [3]:
from pathlib import Path

def load_config_params(start: Path, appliance_name: Optional[str] = None, appliance_index: int = 0) -> Dict[str, Any]:
    def _find_repo_root(start_path: Path) -> Path:
        for p in [start_path] + list(start_path.parents):
            if (p / "config.yaml").exists():
                return p
        return start_path.parent

    repo_root = _find_repo_root(start)
    config_path = repo_root / "config.yaml"
    with open(config_path, "r") as fh:
        cfg = yaml.safe_load(fh) or {}

    apps = cfg.get("appliances") or []
    selected = None
    if isinstance(apps, list) and apps:
        if appliance_name:
            selected = next((a for a in apps if a.get("name") == appliance_name), None)
            if selected is None:
                raise ValueError(f"No appliance named '{appliance_name}' in config.yaml")
        else:
            try:
                selected = apps[int(appliance_index)]
            except Exception:
                selected = apps[0]

    target = ((selected or {}).get("target") or cfg.get("target") or "").strip()
    if not target:
        raise ValueError('config.yaml missing "target"')

    token = (((selected or {}).get("token") or cfg.get("token") or "").strip())
    token_file = (selected or {}).get("token_file") or cfg.get("token_file") or cfg.get("f_token")
    if not token and token_file:
        tf_path = Path(token_file)
        if not tf_path.is_absolute():
            tf_path = repo_root / tf_path
        with open(tf_path, "r") as tf:
            token = tf.read().strip()
    if not token:
        raise ValueError("API token not found in config.yaml (token or token_file)")

    api_version = str((selected or {}).get("api_version") or cfg.get("api_version") or "v1.14")
    verify_ssl = bool((selected or {}).get("verify_ssl", cfg.get("verify_ssl", True)))

    sanitized = target.replace('.', '_').replace(':', '_').replace('/', '_')
    output_dir = repo_root / f"output_{sanitized}"
    output_dir.mkdir(parents=True, exist_ok=True)

    return {
        "repo_root": repo_root,
        "config_path": config_path,
        "cfg": cfg,
        "selected": selected,
        "target": target,
        "token": token,
        "api_version": api_version,
        "verify_ssl": verify_ssl,
        "output_dir": output_dir,
    }


In [4]:
def init_appliance(appliance_name: Optional[str] = "prod") -> Dict[str, Any]:
    params = load_config_params(Path.cwd(), appliance_name=appliance_name)
    target = params["target"]
    api_number = params["api_version"].lstrip('v')

    print('Base Host     :', target)
    print('API Version   :', api_number)
    print('Verify SSL    :', params["verify_ssl"])
    print('Output folder :', params["output_dir"])

    app = tideway.appliance(target, params["token"], api_version=api_number, ssl_verify=params["verify_ssl"])
    twsearch = app.data()

    try:
        about = app.api_about
        print('Appliance reachable:', getattr(about, 'status_code', 'ok'))
    except Exception as exc:
        print('Warning: failed to contact appliance /api/about:', exc)

    return {
        "params": params,
        "target": target,
        "app": app,
        "search": twsearch,
        "api_version": api_number,
        "output_dir": params["output_dir"],
        "name": (params["selected"] or {}).get("name") or (appliance_name or target),
    }


In [5]:
def load_query(title: str) -> str:
    xml_path = Path('../queries/dismal_queries.xml')
    tree = ET.parse(xml_path)
    root = tree.getroot()
    for query in root.findall('query'):
        if query.get('title') == title:
            search = query.find('search')
            if search is not None and search.text:
                return search.text
    raise ValueError(f"Query '{title}' not found in dismal_queries.xml")

qry_active_runs = load_query('Discovery Run Analysis')


In [6]:
instances: list[Dict[str, Any]] = []

def _attempt_init(name: Optional[str]):
    label = name if name else 'default'
    try:
        print(f"Initialise {label.capitalize()}:")
        inst = init_appliance(name)
        instances.append(inst)
    except Exception as exc:
        print(f"Skipping {label}: {exc}")

_attempt_init('prod')
_attempt_init('dev')

if not instances:
    _attempt_init(None)

if not instances:
    raise RuntimeError('No appliances could be initialised from config.yaml')


Initialise Prod:
Base Host     : geodisams-disc-itom.onbmc.com
API Version   : 1.14
Verify SSL    : True
Output folder : /Users/fitzmoskal/Documents/GitHub/DisMAL/output_geodisams-disc-itom_onbmc_com
Appliance reachable: 200
Initialise Dev:
Base Host     : geodisams-disc-itom-dev.onbmc.com
API Version   : 1.14
Verify SSL    : True
Output folder : /Users/fitzmoskal/Documents/GitHub/DisMAL/output_geodisams-disc-itom-dev_onbmc_com
Appliance reachable: 200


In [7]:
def fetch_active_runs(instance: Dict[str, Any]) -> pd.DataFrame:
    search = instance['search']
    try:
        results = search.search({'query': qry_active_runs}, format='object', limit=200)
    except Exception as exc:
        print(f"TW search failed for {instance['target']}: {exc}")
        results = []

    df = pd.DataFrame(results) if results else pd.DataFrame()
    df.insert(0, 'Discovery Instance', instance['target'])

    column_map = {
        'Scan Label': 'Label',
        'End Time': 'End Time',
        'Explicit Ranges': 'Explicit Ranges',
        'Outpost Name': 'Outpost',
        'Scan Level': 'Scan Level',
        'Scan Type': 'Scan Type',
        'Range': 'Range Summary',
        'Total Endpoints': 'Total Endpoints',
        'Active Endpoints': 'Active Endpoints',
        'Dropped': 'Dropped',
        'Scan Kinds': 'Scan Kinds',
    }
    df = df.rename(columns=column_map)

    if not df.empty:
        if 'End Time' in df.columns:
            df['finished'] = df['End Time'].notna()
        else:
            df['finished'] = pd.NA
    else:
        df['finished'] = pd.Series(dtype='boolean')
    return df


def convert_numeric_columns(frame: pd.DataFrame) -> pd.DataFrame:
    numeric_columns = ['Total Endpoints', 'Active Endpoints', 'Dropped']
    converted = frame.copy()
    for col in numeric_columns:
        if col in converted.columns:
            converted[col] = pd.to_numeric(converted[col], errors='coerce').astype('Int64')
    return converted


## Fetch discovery runs

Call the Discovery API endpoint that lists discovery runs. We normalize the JSON into a pandas DataFrame.

In [8]:
runs_by_instance: list[dict[str, Any]] = []
for inst in instances:
    df_runs = fetch_active_runs(inst)
    runs_by_instance.append({'instance': inst, 'data': df_runs})
    print(inst['target'])
    if not df_runs.empty:
        display(df_runs.head(10))
    else:
        print('No runs returned.')


geodisams-disc-itom.onbmc.com


,Discovery Instance,valid_ranges,label,End Time,range_summary,outpost_name,Label,Scan Kind,Range Summary,Schedule,total,Active Endpoints,Dropped,Scan Kinds,finished
0,geodisams-disc-itom.onbmc.com,10.212.66.0/24,FDC Dev VMware,2025-08-29T19:03:55.584694+00:00,10.212.66.0/24,FDC00HELA610W,FDC Dev VMware,IP,[10.212.66.0/24],"Every day, starting at midnight and noon until...",505,266,239,[IP],True
1,geodisams-disc-itom.onbmc.com,10.202.225.0/24,PDC - FF_Servers,2025-08-29T16:57:01.173857+00:00,10.202.225.0/24,PDC00HELA611W,PDC - FF_Servers,IP,[10.202.225.0/24],"Every day, starting at midnight and noon until...",254,23,231,IP,True
2,geodisams-disc-itom.onbmc.com,10.202.110.0/24,PDC - Infrastructure_Applications,2025-08-29T19:26:01.592773+00:00,10.202.110.0/24,PDC00HELA610W,PDC - Infrastructure_Applications,IP,[10.202.110.0/24],"Every day, starting at 2:00 and 14:00 until co...",1609,1609,0,IP,True
3,geodisams-disc-itom.onbmc.com,10.212.70.0/24,FDC - F5_DG_Dev,2025-08-29T19:02:36.851610+00:00,10.212.70.0/24,FDC00HELA610W,FDC - F5_DG_Dev,IP,[10.212.70.0/24],"Every day, starting at 3:00 and 15:00 until co...",254,2,252,IP,True
4,geodisams-disc-itom.onbmc.com,10.212.172.0/23,FDC - DEV_Manhattan_Environments,2025-08-29T19:07:18.928940+00:00,10.212.172.0/23,FDC00HELA610W,FDC - DEV_Manhattan_Environments,IP,[10.212.172.0/23],"Every day, starting at 3:00 and 15:00 until co...",510,14,496,[IP],True
5,geodisams-disc-itom.onbmc.com,10.202.82.0/23,PDC - CITRIX PROD ENV,2025-08-30T02:59:03.550434+00:00,10.202.82.0/23,PDC00HELA610W,PDC - CITRIX PROD ENV,IP,[10.202.82.0/23],"Every day, starting at 6:00 and 18:00 until co...",510,360,150,IP,True
6,geodisams-disc-itom.onbmc.com,10.212.66.0/24,FDC Dev VMware,2025-08-30T02:57:00.590359+00:00,10.212.66.0/24,FDC00HELA610W,FDC Dev VMware,IP,[10.212.66.0/24],"Every day, starting at midnight and noon until...",505,266,239,[IP],True
7,geodisams-disc-itom.onbmc.com,10.202.225.0/24,PDC - FF_Servers,2025-08-30T01:33:36.717255+00:00,10.202.225.0/24,PDC00HELA611W,PDC - FF_Servers,IP,[10.202.225.0/24],"Every day, starting at midnight and noon until...",254,23,231,IP,True
8,geodisams-disc-itom.onbmc.com,10.202.82.0/23,PDC - CITRIX PROD ENV,2025-08-30T03:14:35.683468+00:00,10.202.82.0/23,PDC00HELA610W,PDC - CITRIX PROD ENV,IP,[10.202.82.0/23],"Every day, starting at 1:00 and 13:00 until co...",510,338,172,[IP],True
9,geodisams-disc-itom.onbmc.com,10.202.105.0/24,PDC - Citrix4.0,2025-08-30T02:11:40.885931+00:00,10.202.105.0/24,PDC00HELA610W,PDC - Citrix4.0,IP,[10.202.105.0/24],"Every day, starting at 1:00 and 13:00 until co...",254,15,239,IP,True


geodisams-disc-itom-dev.onbmc.com


,Discovery Instance,valid_ranges,label,End Time,range_summary,outpost_name,Label,Scan Kind,Range Summary,Schedule,total,Active Endpoints,Dropped,Scan Kinds,finished
0,geodisams-disc-itom-dev.onbmc.com,"10.142.169.0/24, 172.20.27.192/27, 172.16.17.8...",FON2 - Subnet Discovery,2025-10-08T07:13:30.576456+00:00,"10.142.169.0/24, 172.20.27.192/27, 172.16.17.8...",None,FON2 - Subnet Discovery,IP,"[10.142.169.0/24, 172.20.27.192/27, 172.16.17....","Every week on Monday, Tuesday, Wednesday and T...",1864,446,1418,[IP],True
1,geodisams-disc-itom-dev.onbmc.com,"10.144.11.0/25, 10.144.11.128/25",AR-BUE1 - Subnet Discovery,2025-10-08T07:16:28.288540+00:00,"10.144.11.0/25, 10.144.11.128/25",None,AR-BUE1 - Subnet Discovery,IP,"[10.144.11.0/25, 10.144.11.128/25]","Every week on Monday, Tuesday, Wednesday and T...",252,43,209,[IP],True
2,geodisams-disc-itom-dev.onbmc.com,10.144.23.240/28,BR-RIO2 - Subnet Discovery,2025-10-08T07:00:03.961502+00:00,10.144.23.240/28,None,BR-RIO2 - Subnet Discovery,IP,[10.144.23.240/28],"Every week on Monday, Tuesday, Wednesday and T...",14,0,14,None,True
3,geodisams-disc-itom-dev.onbmc.com,10.144.41.0/24,CH-SCL2 - Subnet Discovery,2025-10-08T07:08:39.470514+00:00,10.144.41.0/24,None,CH-SCL2 - Subnet Discovery,IP,[10.144.41.0/24],"Every week on Monday, Tuesday, Wednesday and T...",254,12,242,[IP],True
4,geodisams-disc-itom-dev.onbmc.com,"10.144.50.0/26, 10.144.50.192/27, 10.144.50.22...",CO-CTG1 - Subnet Discovery,2025-10-08T07:01:37.168833+00:00,"10.144.50.0/26, 10.144.50.192/27, 10.144.50.22...",None,CO-CTG1 - Subnet Discovery,IP,"[10.144.50.0/26, 10.144.50.192/27, 10.144.50.2...","Every week on Monday, Tuesday, Wednesday and T...",122,7,115,[IP],True
5,geodisams-disc-itom-dev.onbmc.com,"10.143.73.0/24, 172.20.30.64/27, 172.16.18.144...",DAL23 - Subnet Discovery,2025-10-08T07:39:12.216282+00:00,"10.143.73.0/24, 172.20.30.64/27, 172.16.18.144...",None,DAL23 - Subnet Discovery,IP,"[10.143.73.0/24, 172.20.30.64/27, 172.16.18.14...","Every week on Monday, Tuesday, Wednesday and T...",1662,214,1448,[IP],True
6,geodisams-disc-itom-dev.onbmc.com,"10.139.144.0/24, 10.139.145.0/24, 172.20.15.96...",EWR3 - Subnet Discovery,2025-10-08T07:04:18.913294+00:00,"10.139.144.0/24, 10.139.145.0/24, 172.20.15.96...",None,EWR3 - Subnet Discovery,IP,"[10.139.144.0/24, 10.139.145.0/24, 172.20.15.9...","Every week on Monday, Tuesday, Wednesday and T...",1726,42,1684,[IP],True
7,geodisams-disc-itom-dev.onbmc.com,"10.144.99.0/28, 10.144.99.32/27, 10.144.99.64/...",FGO-BOS1 - Subnet Discovery,2025-10-08T07:01:16.658485+00:00,"10.144.99.0/28, 10.144.99.32/27, 10.144.99.64/...",None,FGO-BOS1 - Subnet Discovery,IP,"[10.144.99.0/28, 10.144.99.32/27, 10.144.99.64...","Every week on Monday, Tuesday, Wednesday and T...",201,16,185,[IP],True
8,geodisams-disc-itom-dev.onbmc.com,"10.148.104.0/26, 10.148.105.0/24, 10.148.106.0...",HOU9 - Subnet Discovery,2025-10-08T07:02:25.791187+00:00,"10.148.104.0/26, 10.148.105.0/24, 10.148.106.0...",None,HOU9 - Subnet Discovery,IP,"[10.148.104.0/26, 10.148.105.0/24, 10.148.106....","Every week on Monday, Tuesday, Wednesday and T...",1924,40,1884,[IP],True
9,geodisams-disc-itom-dev.onbmc.com,"10.140.113.0/24, 172.20.18.224/27, 172.16.12.2...",JOL1 - Subnet Discovery,2025-10-08T07:12:42.033627+00:00,"10.140.113.0/24, 172.20.18.224/27, 172.16.12.2...",None,JOL1 - Subnet Discovery,IP,"[10.140.113.0/24, 172.20.18.224/27, 172.16.12....","Every week on Monday, Tuesday, Wednesday and T...",1674,232,1442,[IP],True


## Inspect common fields

Show a few relevant columns such as labels, timing and counts when present.

In [9]:
for item in runs_by_instance:
    inst = item['instance']
    df_runs = item['data']
    if df_runs.empty:
        print(f"No runs for {inst['target']}")
        continue
    if 'finished' in df_runs.columns:
        in_progress = df_runs[df_runs['finished'] != True]
        print(f"In-progress runs for {inst['target']}: {len(in_progress)}")
        if not in_progress.empty:
            display(in_progress.head(10))
    else:
        print(f"{inst['target']} dataset has no 'finished' indicator.")


In-progress runs for geodisams-disc-itom.onbmc.com: 8


,Discovery Instance,valid_ranges,label,End Time,range_summary,outpost_name,Label,Scan Kind,Range Summary,Schedule,total,Active Endpoints,Dropped,Scan Kinds,finished
3868,geodisams-disc-itom.onbmc.com,"10.143.152.128/26, 10.143.153.0/24, 10.143.156...",OSH1 - Subnet Discovery,None,"10.143.152.128/26, 10.143.153.0/24, 10.143.156...",None,OSH1 - Subnet Discovery,IP,"[10.143.152.128/26, 10.143.153.0/24, 10.143.15...","Every week on Monday, Tuesday, Wednesday and T...",2162,383,1491,[IP],False
3869,geodisams-disc-itom.onbmc.com,"10.137.97.0/24, 10.137.98.0/24, 10.141.24.0/24...",PFD8 - Subnet Discovery,None,"10.137.97.0/24, 10.137.98.0/24, 10.141.24.0/24...",None,PFD8 - Subnet Discovery,IP,"[10.137.97.0/24, 10.137.98.0/24, 10.141.24.0/2...","Every week on Monday, Tuesday, Wednesday and T...",4910,402,2166,[IP],False
3870,geodisams-disc-itom.onbmc.com,"10.142.209.0/24, 172.20.28.96/27, 172.16.17.16...",RVL7 - Subnet Discovery,None,"10.142.209.0/24, 172.20.28.96/27, 172.16.17.16...",None,RVL7 - Subnet Discovery,IP,"[10.142.209.0/24, 172.20.28.96/27, 172.16.17.1...","Every week on Monday, Tuesday, Wednesday and T...",1846,356,1283,[IP],False
3871,geodisams-disc-itom.onbmc.com,"10.136.81.0/24, 172.20.1.0/27, 172.21.10.0/28,...",SPK1 - Subnet Discovery,None,"10.136.81.0/24, 172.20.1.0/27, 172.21.10.0/28,...",None,SPK1 - Subnet Discovery,IP,"[10.136.81.0/24, 172.20.1.0/27, 172.21.10.0/28...","Every week on Monday, Tuesday, Wednesday and T...",2222,307,1780,[IP],False
3883,geodisams-disc-itom.onbmc.com,10.202.66.0/24,PDC Dev VMware,None,10.202.66.0/24,PDC00HELA610W,PDC Dev VMware,IP,[10.202.66.0/24],"Every day, starting at midnight and noon until...",564,315,239,[IP],False
3892,geodisams-disc-itom.onbmc.com,10.202.246.0/24,VLAN246 - NonProd/QA Management VLAN,None,10.202.246.0/24,PDC00HELA610W,VLAN246 - NonProd/QA Management VLAN,IP,[10.202.246.0/24],"Every day, starting at midnight and noon until...",254,4,244,[IP],False
3894,geodisams-disc-itom.onbmc.com,"10.144.40.0/24, 10.144.45.0/24, 10.144.47.0/24...",CH-SCL1 - Subnet Discovery,None,"10.144.40.0/24, 10.144.45.0/24, 10.144.47.0/24...",None,CH-SCL1 - Subnet Discovery,IP,"[10.144.40.0/24, 10.144.45.0/24, 10.144.47.0/2...","Every week on Monday, Tuesday, Wednesday and T...",1016,106,898,[IP],False
3900,geodisams-disc-itom.onbmc.com,"10.154.153.0/24, 10.154.152.16/28, 10.154.152....",IN-MUM1 - Subnet Discovery,None,"10.154.153.0/24, 10.154.152.16/28, 10.154.152....",None,IN-MUM1 - Subnet Discovery,IP,"[10.154.153.0/24, 10.154.152.16/28, 10.154.152...","Every week on Monday, Tuesday, Wednesday and T...",282,104,160,[IP],False


In-progress runs for geodisams-disc-itom-dev.onbmc.com: 10


,Discovery Instance,valid_ranges,label,End Time,range_summary,outpost_name,Label,Scan Kind,Range Summary,Schedule,total,Active Endpoints,Dropped,Scan Kinds,finished
82,geodisams-disc-itom-dev.onbmc.com,"10.140.192.0/24, 10.140.193.0/24, 172.20.20.32...",PFD3 - Subnet Discovery,None,"10.140.192.0/24, 10.140.193.0/24, 172.20.20.32...",None,PFD3 - Subnet Discovery,IP,"[10.140.192.0/24, 10.140.193.0/24, 172.20.20.3...","Every week on Monday, Tuesday, Wednesday and T...",1980,146,1722,[IP],False
83,geodisams-disc-itom-dev.onbmc.com,"10.138.1.0/24, 10.138.2.0/24, 10.138.5.0/24, 1...",SAV4 - Subnet Discovery,None,"10.138.1.0/24, 10.138.2.0/24, 10.138.5.0/24,...",None,SAV4 - Subnet Discovery,IP,"[10.138.1.0/24, 10.138.2.0/24, 10.138.5.0/24, ...","Every week on Monday, Tuesday, Wednesday and T...",1530,178,1274,[IP],False
84,geodisams-disc-itom-dev.onbmc.com,"10.141.224.0/24, 10.141.225.0/24, 10.141.226.0...",TRO1 - Subnet Discovery,None,"10.141.224.0/24, 10.141.225.0/24, 10.141.226.0...",None,TRO1 - Subnet Discovery,IP,"[10.141.224.0/24, 10.141.225.0/24, 10.141.226....","Every week on Monday, Tuesday, Wednesday and T...",1346,97,1248,[IP],False
85,geodisams-disc-itom-dev.onbmc.com,"10.136.225.0/24, 172.20.4.128/27, 172.16.5.192...",RIV2 - Subnet Discovery,None,"10.136.225.0/24, 172.20.4.128/27, 172.16.5.192...",None,RIV2 - Subnet Discovery,IP,"[10.136.225.0/24, 172.20.4.128/27, 172.16.5.19...","Every week on Monday, Tuesday, Wednesday and T...",1768,175,1513,[IP],False
86,geodisams-disc-itom-dev.onbmc.com,"10.154.136.0/27, 10.154.136.32/27, 10.154.136....",IN-CHE2,None,"10.154.136.0/27, 10.154.136.32/27, 10.154.136....",None,IN-CHE2,IP,"[10.154.136.0/27, 10.154.136.32/27, 10.154.136...","Every day, starting at midnight and noon until...",668,171,270,[IP],False
87,geodisams-disc-itom-dev.onbmc.com,"10.154.144.0/27, 10.154.144.32/28, 10.154.144....",IN-CHE3,None,"10.154.144.0/27, 10.154.144.32/28, 10.154.144....",None,IN-CHE3,IP,"[10.154.144.0/27, 10.154.144.32/28, 10.154.144...","Every day, starting at midnight and noon until...",634,123,419,[IP],False
88,geodisams-disc-itom-dev.onbmc.com,"10.143.41.0/24, 172.20.29.192/27, 172.16.18.80...",CMH1 - Subnet Discovery,None,"10.143.41.0/24, 172.20.29.192/27, 172.16.18.80...",None,CMH1 - Subnet Discovery,IP,"[10.143.41.0/24, 172.20.29.192/27, 172.16.18.8...","Every week on Monday, Tuesday, Wednesday and T...",1668,163,1394,[IP],False
90,geodisams-disc-itom-dev.onbmc.com,"10.138.233.0/24, 172.20.12.192/27, 172.16.9.20...",DAL7 - Subnet Discovery,None,"10.138.233.0/24, 172.20.12.192/27, 172.16.9.20...",None,DAL7 - Subnet Discovery,IP,"[10.138.233.0/24, 172.20.12.192/27, 172.16.9.2...","Every week on Monday, Tuesday, Wednesday and T...",1700,153,1432,[IP],False
93,geodisams-disc-itom-dev.onbmc.com,"10.154.148.16/28, 10.154.149.0/24, 10.154.148....",IN-GUR1 - Subnet Discovery,None,"10.154.148.16/28, 10.154.149.0/24, 10.154.148....",None,IN-GUR1 - Subnet Discovery,IP,"[10.154.148.16/28, 10.154.149.0/24, 10.154.148...","Every week on Monday, Tuesday, Wednesday and T...",310,106,197,[IP],False
94,geodisams-disc-itom-dev.onbmc.com,"10.137.240.0/24, 10.137.241.0/24, 172.20.8.224...",LAX5 - Subnet Discovery,None,"10.137.240.0/24, 10.137.241.0/24, 172.20.8.224...",None,LAX5 - Subnet Discovery,IP,"[10.137.240.0/24, 10.137.241.0/24, 172.20.8.22...","Every week on Monday, Tuesday, Wednesday and T...",1726,118,1607,[IP],False


## Filter in-progress runs

Filter the DataFrame to show only runs that are not yet finished (when the `finished` field is present).

In [10]:
processed_runs: list[dict[str, Any]] = []
for item in runs_by_instance:
    inst = item['instance']
    df_runs = convert_numeric_columns(item['data'])
    other_cols = [c for c in df_runs.columns if c != 'Discovery Instance']
    if other_cols:
        df_runs = df_runs[['Discovery Instance'] + other_cols]
    processed_runs.append({'instance': inst, 'data': df_runs})

if processed_runs:
    combined_df = pd.concat([item['data'] for item in processed_runs], ignore_index=True)
else:
    combined_df = pd.DataFrame(columns=['Discovery Instance'])

display(combined_df.head(10))


,Discovery Instance,valid_ranges,label,End Time,range_summary,outpost_name,Label,Scan Kind,Range Summary,Schedule,total,Active Endpoints,Dropped,Scan Kinds,finished
0,geodisams-disc-itom.onbmc.com,10.212.66.0/24,FDC Dev VMware,2025-08-29T19:03:55.584694+00:00,10.212.66.0/24,FDC00HELA610W,FDC Dev VMware,IP,[10.212.66.0/24],"Every day, starting at midnight and noon until...",505,266,239,[IP],True
1,geodisams-disc-itom.onbmc.com,10.202.225.0/24,PDC - FF_Servers,2025-08-29T16:57:01.173857+00:00,10.202.225.0/24,PDC00HELA611W,PDC - FF_Servers,IP,[10.202.225.0/24],"Every day, starting at midnight and noon until...",254,23,231,IP,True
2,geodisams-disc-itom.onbmc.com,10.202.110.0/24,PDC - Infrastructure_Applications,2025-08-29T19:26:01.592773+00:00,10.202.110.0/24,PDC00HELA610W,PDC - Infrastructure_Applications,IP,[10.202.110.0/24],"Every day, starting at 2:00 and 14:00 until co...",1609,1609,0,IP,True
3,geodisams-disc-itom.onbmc.com,10.212.70.0/24,FDC - F5_DG_Dev,2025-08-29T19:02:36.851610+00:00,10.212.70.0/24,FDC00HELA610W,FDC - F5_DG_Dev,IP,[10.212.70.0/24],"Every day, starting at 3:00 and 15:00 until co...",254,2,252,IP,True
4,geodisams-disc-itom.onbmc.com,10.212.172.0/23,FDC - DEV_Manhattan_Environments,2025-08-29T19:07:18.928940+00:00,10.212.172.0/23,FDC00HELA610W,FDC - DEV_Manhattan_Environments,IP,[10.212.172.0/23],"Every day, starting at 3:00 and 15:00 until co...",510,14,496,[IP],True
5,geodisams-disc-itom.onbmc.com,10.202.82.0/23,PDC - CITRIX PROD ENV,2025-08-30T02:59:03.550434+00:00,10.202.82.0/23,PDC00HELA610W,PDC - CITRIX PROD ENV,IP,[10.202.82.0/23],"Every day, starting at 6:00 and 18:00 until co...",510,360,150,IP,True
6,geodisams-disc-itom.onbmc.com,10.212.66.0/24,FDC Dev VMware,2025-08-30T02:57:00.590359+00:00,10.212.66.0/24,FDC00HELA610W,FDC Dev VMware,IP,[10.212.66.0/24],"Every day, starting at midnight and noon until...",505,266,239,[IP],True
7,geodisams-disc-itom.onbmc.com,10.202.225.0/24,PDC - FF_Servers,2025-08-30T01:33:36.717255+00:00,10.202.225.0/24,PDC00HELA611W,PDC - FF_Servers,IP,[10.202.225.0/24],"Every day, starting at midnight and noon until...",254,23,231,IP,True
8,geodisams-disc-itom.onbmc.com,10.202.82.0/23,PDC - CITRIX PROD ENV,2025-08-30T03:14:35.683468+00:00,10.202.82.0/23,PDC00HELA610W,PDC - CITRIX PROD ENV,IP,[10.202.82.0/23],"Every day, starting at 1:00 and 13:00 until co...",510,338,172,[IP],True
9,geodisams-disc-itom.onbmc.com,10.202.105.0/24,PDC - Citrix4.0,2025-08-30T02:11:40.885931+00:00,10.202.105.0/24,PDC00HELA610W,PDC - Citrix4.0,IP,[10.202.105.0/24],"Every day, starting at 1:00 and 13:00 until co...",254,15,239,IP,True


## Save to CSV (optional)

Persist the full dataset to the project output directory (`output_<target>`).

This cell formats the output to match the DisMAL CLI report for Active Scans by:
- Inserting a 'Discovery Instance' column as the first column.
- Casting numeric fields (done, pre_scanning, scanning, total) to integers when present.
- Sorting remaining columns alphabetically to mirror json2csv header ordering.

In [11]:
for item in processed_runs:
    inst = item['instance']
    df_runs = item['data']
    output_csv = inst['output_dir'] / 'active_scans.csv'
    df_runs.to_csv(output_csv, index=False)
    print(f'Saved to {output_csv}')


Saved to /Users/fitzmoskal/Documents/GitHub/DisMAL/output_geodisams-disc-itom_onbmc_com/active_scans.csv
Saved to /Users/fitzmoskal/Documents/GitHub/DisMAL/output_geodisams-disc-itom-dev_onbmc_com/active_scans.csv


---
### Notes
- If your appliance uses a self-signed certificate, set `VERIFY_SSL = False`.
- If the appliance exposes a different API version, update `API_VERSION`.
- You can further transform the dataset with `pandas.json_normalize` or additional joins if needed.